## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,74.39,75,3,15.12,clear sky
1,1,Chuy,UY,-33.6971,-53.4616,58.42,91,100,3.36,overcast clouds
2,2,Mar Del Plata,AR,-38.0023,-57.5575,52.27,72,3,7.31,clear sky
3,3,Antofagasta,CL,-23.6500,-70.4000,56.48,63,35,7.74,scattered clouds
4,4,Kapaa,US,22.0752,-159.3190,73.38,79,11,12.08,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your minimum desired temperature for your trip? "))
max_temp = float(input("What is your maximum desired temperature for your trip? "))

What is your minimum desired temperature for your trip? 45
What is your maximum desired temperature for your trip? 60


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Chuy,UY,-33.6971,-53.4616,58.42,91,100,3.36,overcast clouds
2,2,Mar Del Plata,AR,-38.0023,-57.5575,52.27,72,3,7.31,clear sky
3,3,Antofagasta,CL,-23.6500,-70.4000,56.48,63,35,7.74,scattered clouds
11,11,Busselton,AU,-33.6500,115.3333,53.10,84,0,8.72,clear sky
23,23,Okha,RU,53.5739,142.9478,55.26,83,53,9.53,broken clouds
...,...,...,...,...,...,...,...,...,...,...
676,676,Vanavara,RU,60.3400,102.2797,55.49,96,45,5.53,scattered clouds
684,684,Porosozero,RU,62.7191,32.7270,59.45,87,67,12.24,light rain
688,688,Maniitsoq,GL,65.4167,-52.9000,45.50,96,100,6.98,overcast clouds
692,692,Mezen,RU,65.8522,44.2400,59.77,93,100,12.24,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Chuy,UY,-33.6971,-53.4616,58.42,91,100,3.36,overcast clouds
2,2,Mar Del Plata,AR,-38.0023,-57.5575,52.27,72,3,7.31,clear sky
3,3,Antofagasta,CL,-23.6500,-70.4000,56.48,63,35,7.74,scattered clouds
11,11,Busselton,AU,-33.6500,115.3333,53.10,84,0,8.72,clear sky
23,23,Okha,RU,53.5739,142.9478,55.26,83,53,9.53,broken clouds
...,...,...,...,...,...,...,...,...,...,...
676,676,Vanavara,RU,60.3400,102.2797,55.49,96,45,5.53,scattered clouds
684,684,Porosozero,RU,62.7191,32.7270,59.45,87,67,12.24,light rain
688,688,Maniitsoq,GL,65.4167,-52.9000,45.50,96,100,6.98,overcast clouds
692,692,Mezen,RU,65.8522,44.2400,59.77,93,100,12.24,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Chuy,UY,58.42,overcast clouds,-33.6971,-53.4616,
2,Mar Del Plata,AR,52.27,clear sky,-38.0023,-57.5575,
3,Antofagasta,CL,56.48,scattered clouds,-23.6500,-70.4000,
11,Busselton,AU,53.10,clear sky,-33.6500,115.3333,
23,Okha,RU,55.26,broken clouds,53.5739,142.9478,
25,Mataura,NZ,48.47,scattered clouds,-46.1927,168.8643,
27,Tigil,RU,51.39,broken clouds,57.8000,158.6667,
32,Hay River,CA,51.15,overcast clouds,60.8156,-115.7999,
33,Vardo,NO,49.15,broken clouds,70.3705,31.1107,
35,Tuktoyaktuk,CA,47.91,broken clouds,69.4541,-133.0374,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")
        

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [17]:
# 7 end with KeyError
#clean_df = clean_df[clean_df["Hotel Name"] != ""]
#clean_df

In [16]:
# 7. Drop the rows where there is no Hotel Name.
cleaning = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = cleaning.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Chuy,UY,58.42,overcast clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
2,Mar Del Plata,AR,52.27,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
3,Antofagasta,CL,56.48,scattered clouds,-23.6500,-70.4000,"Holiday Inn Express Antofagasta, an IHG Hotel"
11,Busselton,AU,53.10,clear sky,-33.6500,115.3333,Observatory Guest House
23,Okha,RU,55.26,broken clouds,53.5739,142.9478,Империя
...,...,...,...,...,...,...,...
658,Colac,AU,51.62,overcast clouds,-38.3333,143.5833,Otway Gate Motel
671,Nome,US,48.16,overcast clouds,64.5011,-165.4064,Dredge 7 Inn
684,Porosozero,RU,59.45,light rain,62.7191,32.7270,Porosozero
688,Maniitsoq,GL,45.50,overcast clouds,65.4167,-52.9000,Hotel Heilmann Lyberth


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))